In [33]:
import networkx as nx
from pyvis import network as net
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from collections import Counter 
import openpyxl as xl
from cascadingfailureanalysis import *

In [34]:
g = nx.read_gml("networks/mg_net.gml")

In [35]:
d,r,_ = cascade_failure(g,'c_MG_356_MONOMER',target_reaction='cellular_division_reaction')
print(d,r)

4 False


In [36]:
replication_reaction = 'cellular_division_reaction'
print(list(set([d['moltype'] for _,d in g.nodes(data=True) if d['type']=='m'])))
genes = {}
for n,d in g.nodes(data=True):
    if d['type']=='m':
        if d['moltype'] == "Imature Protein Monomer":
            gn = n.replace("_MONOMER","").replace("Imature_","")[2:]
            genes[gn] = n
        if d['moltype']=="RNA" and ("tRNA" in d['annotations'] or "rRNA" in d['annotations'] or "sRNA" in d['annotations']):
            gn = n[2:]
            genes[gn] = n
            
for n,m in genes.items():
    print(n,m)
            
wb = xl.load_workbook('mg_essentiality_basis.xlsx')
sh=wb['Sheet1']
for i in range(1,sh.max_row):
    gene = sh['A'+str(i+1)].value
    if gene in genes.keys():        
        sh['C'+str(i+1)].value = genes[gene]
wb.save('mg_essentiality.xlsx')

['Peptide', 'Protein Monomer', 'Stimulus', 'Translation Complex', 'Replication Complex', 'DNA-Protein Complex', 'Imature Protein Monomer', 'RNA', 'DNA', 'Transcription Complex', 'Protein Complex', 'RNA Complex', 'Small Molecule']
MG471 c_MG471
MG483 c_MG483
MG472 c_MG472
MG488 c_MG488
MG493 c_MG493
MG501 c_MG501
MG489 c_MG489
MG492 c_MG492
MG499 c_MG499
MG490 c_MG490
MG511 c_MG511
MG502 c_MG502
MG518 c_MG518
MG475 c_MG475
MG523 c_MG523
MG513 c_MG513
MG497 c_MG497
MG496 c_MG496
MG510 c_MG510
MG507 c_MG507
MG487 c_MG487
MG519 c_MG519
MG509 c_MG509
MG495 c_MG495
MG500 c_MG500
MG_0004 c_MG_0004
MG484 c_MG484
MG520 c_MG520
MG503 c_MG503
MG514 c_MG514
MG479 c_MG479
MG506 c_MG506
MG485 c_MG485
MG508 c_MG508
MG512 c_MG512
MG486 c_MG486
MG504 c_MG504
MG_344 c_Imature_MG_344_MONOMER
MG_245 c_Imature_MG_245_MONOMER
MG_517 c_Imature_MG_517_MONOMER
MG_310 c_Imature_MG_310_MONOMER
MG_327 c_Imature_MG_327_MONOMER
MG_212 c_Imature_MG_212_MONOMER
MG_071 c_Imature_MG_071_MONOMER
MG_300 c_Imature_MG_300_

In [37]:
gene_is_essential = {n:cascade_failure(g,m,target_reaction=replication_reaction)[1] for n,m in genes.items()}

In [38]:
label = {True:"Y",False:"N"}

for i in range(1,sh.max_row):
    gene = sh['A'+str(i+1)].value
    if gene in gene_is_essential.keys():        
        sh['F'+str(i+1)].value = label[gene_is_essential[gene]]
wb.save('mg_essentiality.xlsx')


In [39]:
double_gene_is_essential = gene_is_essential.copy()
double_deletions_essentiality = []
non_essential_genes = [n for n,m in double_gene_is_essential.items() if not m]
print("Non essential genes",len(non_essential_genes))
for i in range(len(non_essential_genes)-1):
    for j in range(i,len(non_essential_genes)):
        g_tuple = [non_essential_genes[i],non_essential_genes[j]]
        m = [genes[g] for g in g_tuple]
        if cascade_failure(g,m,target_reaction=replication_reaction)[1]:
            double_deletions_essentiality.append(g_tuple)
for gs in double_deletions_essentiality:
    double_gene_is_essential[gs[0]] = True
    double_gene_is_essential[gs[1]] = True
print("Pairs of essential genes",len(double_deletions_essentiality))

Non essential genes 322
Pairs of essential genes 20


In [40]:
for i in range(1,sh.max_row):
    gene = sh['A'+str(i+1)].value
    if gene in gene_is_essential.keys():        
        sh['H'+str(i+1)].value = label[double_gene_is_essential[gene]]
wb.save('mg_essentiality.xlsx')

In [43]:
pair_g = nx.Graph()
edges = [tuple(e) for e in double_deletions_essentiality]
pair_g.add_edges_from(edges)
nx.write_gml(pair_g,'networks/mg_double_deletion_essential.gml')
nx.write_graphml(pair_g,'networks/mg_double_deletion_essential.graphml')


In [44]:
pair_g = nx.read_gml('networks/mg_double_deletion_essential.gml')
pyviz_g = net.Network(notebook=True)
pyviz_g.from_nx(pair_g)
pyviz_g.show_buttons(filter_=['physics'])
pyviz_g.show("pair_genes.html")

In [41]:
print(pair_g.nodes())

['MG_245', 'MG_013', 'MG_071', 'MG_324', 'MG_183', 'MG_020', 'MG_239', 'MG_391', 'MG_208', 'MG_046', 'MG_322', 'MG_323']


In [12]:
#fmet_nodes = [n for n in g.nodes() if 'FMET' in n]
#print(fmet_nodes)
#fmet_nodes = ['c_FMET','c_MG488_FMET','c_MG_245_MONOMER','c_MG_013_DIMER']
#fmet_reactions = []
#for m in fmet_nodes:
#    for key in g.successors(m):
#        fmet_reactions.append(key)
#    for key in g.predecessors(m):
#        fmet_reactions.append(key)
#fmet_reactions = list(set([r for r in fmet_reactions if "Degradation" not in r and 'Elongation' not in r]))
#print(fmet_reactions)
fmet_reaction = ['FthC', 'MG488_Formyltransferase', 'FMETHs', 'FTHFS', 'MG_013_DIMER_BIOSYNTHESIS', 'FolD1', 'FMETHs_rev', 'FolD1_rev', 'FthC_rev', 'FolD2', 'FolD2_rev','MG488_FMET_Degradation']
fmet_net_nodes = fmet_reaction[:]
for r in fmet_reaction:
    for key in g.successors(r):
        fmet_net_nodes.append(key)
    for key in g.predecessors(r):
        fmet_net_nodes.append(key)
fmet_net_nodes = list(set(fmet_net_nodes))
fmet_net = g.subgraph(fmet_net_nodes)
nx.write_gml(fmet_net,'networks/mg_fmet_subnet.gml')
nx.write_graphml(fmet_net,'networks/mg_fmet_subnet.graphml')
print(fmet_net_nodes)

['FolD2', 'MG488_FMET_Degradation', 'c_FMET', 'c_MG_083_MONOMER', 'c_CMP', 'c_H', 'FolD1_rev', 'c_THF', 'c_FOR', 'FolD1', 'c_NADP', 'MG488_Formyltransferase', 'MG_013_DIMER_BIOSYNTHESIS', 'c_NADPH', 'c_ATP', 'c_MG488_FMET', 'c_MG488_MET', 'c_GMP', 'c_FTHF10', 'FthC', 'FMETHs', 'c_H2O', 'c_MET', 'c_AMP', 'c_PI', 'FTHFS', 'FolD2_rev', 'FMETHs_rev', 'c_MG_245_MONOMER', 'c_MG_013_DIMER', 'c_UMP', 'c_ADP', 'c_METTHF', 'c_MG_013_MONOMER', 'c_MG_365_MONOMER', 'c_METHF', 'c_MG_104_MONOMER', 'FthC_rev']


In [13]:
#knockout_damage = {}
#knockout_essential = {}
#for node in [m for m in g.nodes()]:
#    d,r,_ = cascade_failure(g,node,target_reaction=replication_reaction)
#    knockout_damage[node] = d
#    knockout_essential[node] = r
#nx.set_node_attributes(g, knockout_damage, 'knockoutDamage')
#nx.set_node_attributes(g, knockout_damage, 'singleKnockoutEssential')

In [14]:
#g = nx.write_gml(g,"mg_net.gml")

In [42]:
net_file = "networks/mg_net.gml"
experimental_file = "mg_essentiality.xlsx"
replication_reaction = 'cellular_division_reaction'
p_step,p_max = '0.02','1.02'
replicates = 10
output_file = "ss.dat"
str_replicates = " ".join([str(i) for i in range(replicates)])

parameters = " ::: ".join([net_file,experimental_file,p_step,p_max,replication_reaction])
with open(output_file,"w") as f:
    f.write("#p\t replica\t essentials\t correct\t fp\t fn\n")
%time !parallel -j10 python3 randomized_gene_essentiality.py ::: {parameters} ::: {str_replicates} >> {output_file}

perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = "en_US:en",
	LC_ALL = (unset),
	LC_TIME = "pt_BR.UTF-8",
	LC_MONETARY = "pt_BR.UTF-8",
	LC_CTYPE = "C.UTF-8",
	LC_ADDRESS = "pt_BR.UTF-8",
	LC_TELEPHONE = "pt_BR.UTF-8",
	LC_NAME = "pt_BR.UTF-8",
	LC_MEASUREMENT = "pt_BR.UTF-8",
	LC_IDENTIFICATION = "pt_BR.UTF-8",
	LC_NUMERIC = "pt_BR.UTF-8",
	LC_PAPER = "pt_BR.UTF-8",
	LANG = "en_US.UTF-8"
    are supported and installed on your system.
perl: warning: Falling back to a fallback locale ("en_US.UTF-8").
perl: warning: Setting locale failed.
perl: warning: Please check that your locale settings:
	LANGUAGE = "en_US:en",
	LC_ALL = (unset),
	LC_TIME = "pt_BR.UTF-8",
	LC_MONETARY = "pt_BR.UTF-8",
	LC_CTYPE = "C.UTF-8",
	LC_ADDRESS = "pt_BR.UTF-8",
	LC_TELEPHONE = "pt_BR.UTF-8",
	LC_NAME = "pt_BR.UTF-8",
	LC_MEASUREMENT = "pt_BR.UTF-8",
	LC_IDENTIFICATION = "pt_BR.UTF-8",
	LC_NUMERIC = "pt_BR.UTF-8",
	LC_PAPER = "pt_BR.UTF-8",
	LANG = "en_